# Laboratorio 3: Problema de tiempo mínimo y lineal cuadrático

**Nombres:** Sebastián Acuña U. y Patricio Yañez A. <br>
**Fecha:** 4 de octubre de 2024 <br>
**Profesor:** Héctor Ramírez C. <br>
**Auxiliar:** Diego Olguín W. <br>
**Ayudantes:** Carlos Antil C. y Luis Fuentes C. <br>
**Curso:** [MA4703-1] Control Óptimo: Teoría y Laboratorio

In [7]:
# Librerías a utilizar

#using Pkg
#Pkg.add("DifferentialEquations")
#Pkg.add("JuMP")
#Pkg.add("NonlinearSolve")
#Pkg.add("OptimalControl")
#Pkg.add("LaTeXStrings")
#Pkg.add("NLPModelsIpopt")
#Pkg.add("ControlSystems")
#Pkg.add("Interpolations")

using LinearAlgebra
using Plots
using LaTeXStrings
using JuMP
using Ipopt
using NLPModelsIpopt
using DifferentialEquations
using NonlinearSolve
using OptimalControl
using ControlSystems
using Interpolations

## Parte A: Control de un carro-cohete en tiempo mínimo y método de resolución directo

### Ejercicio 1

Tenemos la ecuación de la dinámica

$$
\ddot{x} = -\alpha x + u
$$

con condiciones de borde $\mathbf{x_0} = (x_0, v_0) = (0,0)$ y $\mathbf{x_{t_f}} = (x_{t_f}, v_{t_f}) = (x_f, 0)$. Nos interesa resolver el problema 

$$
\min_{u \in \mathcal{A}}{t_f}; \ \ x(t_f;u,\mathbf{x_0}) = x_f; \ \ v(t_f;u,\mathbf{x_0}) = 0
$$

donde $|u(t)| \leq 1, \; \forall t$. 

Definiendo $v = \dot{x}$ se genera un sistema de ecuaciones cuya forma matricial es

$$
\dot{\begin{pmatrix}
x(t) \\
v(t)
\end{pmatrix}}
=
\begin{pmatrix}
0 & 1 \\
-\alpha & 0
\end{pmatrix}
\begin{pmatrix}
x(t) \\
v(t)
\end{pmatrix}
+
\begin{pmatrix}
0 \\
1
\end{pmatrix}
u(t)
$$

Si discretizamos la dinámica particionando el intervalo $[0,t_f]$ en $N$ puntos $0 = t_1 < t_2 < \cdots < t_{N} = t_f$, donde $t_i = (i-1) \dfrac{t_f}{N-1}$ obtenemos el siguiente sistema 

\begin{align*}
\dfrac{x_{i+1}-x_i}{h} &= v_i \\
\dfrac{v_{i+1}-v_i}{h} &= -\alpha x_i + u_i
\end{align*}

donde $x_i = x(t_i), v_i = v(t_i), u_i = u(t_i)$ y $h = \frac{t_f}{N-1}$. Esto en particular implica que 

$$
\begin{pmatrix}
x_{i+1} \\
v_{i+1}
\end{pmatrix} = \begin{pmatrix}
x_i \\
v_i
\end{pmatrix} + h \begin{pmatrix}
v_i \\
-\alpha x_i
\end{pmatrix} + h \begin{pmatrix}
0 \\
u_i
\end{pmatrix}
$$

Con ello, el problema de optimización consiste en minimizar $t_f$ bajo las restricciones provistas por la dinámica discretizada, el valor de $h$ como *step function* y las condiciones de borde mencionadas al inicio.

### Ejercicio 2

In [8]:
# Definición de la discretización de Euler y resolución utilizando JuMP

function euler_discretization(N, α, x_f)
    model = JuMP.Model(Ipopt.Optimizer)
    
    @variable(model, 0 <= t_f)
    @variable(model, -1 <= u[1:N-1] <= 1)
    @variable(model, x[1:N])
    @variable(model, v[1:N])
    
    h = t_f / (N - 1)
    
    @constraint(model, x[1] == 0)
    @constraint(model, v[1] == 0)
    @constraint(model, x[N] == x_f)
    @constraint(model, v[N] == 0)
    
    for i in 1:N-1
        @constraint(model, x[i+1] == x[i] + h * v[i])
        @constraint(model, v[i+1] == v[i] + h * (-α * x[i] + u[i]))
    end
    
    @objective(model, Min, t_f)
    
    optimize!(model)
    
    return value.(x), value.(v), value.(u), value(t_f)
end

# Parámetros de juguete
N = 100
α = 1.0
x_f = 100.0

# Resolución
x, v, u, t_f = euler_discretization(N, α, x_f)

println("Tiempo final óptimo: ", t_f)
println("Posiciones: ", x)
println("Velocidades: ", v)
println("Control: ", u)

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      994
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      297

Total number of variables............................:      300
                     variables with only lower bounds:        1
                variables with lower and upper bounds:       99
                     variables with only upper bounds:        0
Total number of equality constraints.................:      202
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9999900e-03 1.00e+02 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [ ]:
# Parámetros del problema
N = 100
α = 5.0
x_f = -20.0

# Resolución
x, v, u, t_f = euler_discretization(N, α, x_f)
t = range(0, stop=t_f, length=N)

In [9]:
# Gráfico de la trayectoria x(t)
plot(t, x, label="Posición x(t)", xlabel="Tiempo t", ylabel="Posición x", title="Trayectoria óptima para N=$N, α=$α, x_f=$x_f")
#display(plot)

LoadError: UndefVarError: `t` not defined

In [10]:
# Gráfico de la velocidad v(t)
plot(t, v, label="Velocidad v(t)", xlabel="Tiempo t", ylabel="Velocidad v", title="Velocidad óptima para N=$N, α=$α, x_f=$x_f")
#display(plot)

LoadError: UndefVarError: `t` not defined

In [11]:
# Gráfico del control u(t)
plot(t[1:N-1], u, label="Control u(t)", xlabel="Tiempo t", ylabel="Control u", title="Control óptimo para N=$N, α=$α, x_f=$x_f", legend=:bottomright)
#display(plot)

LoadError: UndefVarError: `t` not defined

In [ ]:
# Distintos valores de N, α y x_f
N_values = [10, 100, 1000]
α_values = [0, 1, 5]
x_f_values = [-20, 100, 4000]

# Resolver el problema de tiempo mínimo discretizado probando las combinaciones de N, α, x_f
results = []

for n in N_values
    for a in α_values
        for xf in x_f_values
            try
                x, v, u, t_f = euler_discretization(n, a, xf)
                push!(results, (N=n, α=a, x_f=xf, t_f=t_f, x=x, v=v, u=u))
                println("N: $n, α: $a, x_f: $xf, t_f: $t_f")
            catch e
                println("Error con N: $n, α: $a, x_f: $xf - ", e)
            end
        end
    end
end

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      796
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      198

Total number of variables............................:      300
                     variables with only lower bounds:        1
                variables with lower and upper bounds:       99
                     variables with only upper bounds:        0
Total number of equality constraints.................:      202
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9999900e-03 2.00e+01 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [12]:
# Graficar la trayectoria y el control óptimo
for result in results
    n, a, xf, t_f, x, v, u = result.N, result.α, result.x_f, result.t_f, result.x, result.v, result.u
    t = range(0, stop=t_f, length=n)
    
    plot(t, x, label="Posición x(t)", xlabel="Tiempo t", ylabel="Posición x", title="Trayectoria óptima para N=$n, α=$a, x_f=$xf")
    plot!(t, v, label="Velocidad v(t)", xlabel="Tiempo t", ylabel="Velocidad v")
    plot!(t[1:end-1], u, label="Control u(t)", xlabel="Tiempo t", ylabel="Control u", legend=:bottomright)
    
    display(plot)
end

LoadError: UndefVarError: `results` not defined

In [4]:
results

12-element Vector{Any}:
 (N = 100, α = -1, x_f = -3, t_f = 8.95534027221356e-9, x = [-0.00011065037644057512, -0.00022132630014168645, -0.0003320448640627494, -0.00044282320726782856, -0.0005536785335683208, -0.0006646281303477718, -0.0007756893876225665, -0.0008868798173934991, -0.000998217073344798, -0.0011097189709490729  …  -0.014884418768164086, -0.015355654041597016, -0.015874317368516477, -0.016453204390287936, -0.017111279523838466, -0.017878487540082103, -0.018806466171754365, -0.019996787071649952, -0.021698567751761804, -0.02488631271443503], v = [6.107404939389249e-13, 1.2164480942456287e-12, 1.8121441749362295e-12, 2.397824878862453e-12, 2.9734847985108704e-12, 3.5391169701459812e-12, 4.094712866250443e-12, 4.640262386241116e-12, 5.175753845434601e-12, 5.701173962230836e-12  …  8.145521904632225e-12, 7.414824325866836e-12, 6.6414997561202906e-12, 5.82125794935809e-12, 4.948651191642418e-12, 4.016516286140068e-12, 3.014981292453129e-12, 1.9295032250571543e-12, 7.36351131631

### Ejercicio 3

In [19]:
# Definición y resolución del problema de tiempo mínimo utilizando OptimalControl

function solve_min_time_continuous(α, x_f)
    model = OptimalControlModel(Ipopt.Optimizer, 2, 1)

    @def begin
        t_f ∈ R, variable
        t ∈ [0, t_f], time
        x ∈ R², state
        u ∈ [-1, 1], control
        x' = [x[2], -α * x[1] + u]
        x(0) = [0, 0]
        x(t_f) = [x_f, 0]
        t_f >= 0
    end

    @objective(model, Min, tf)

    optimize!(model)

    return solution(model)
end

# Parámetros de juguete
α = 1.0
x_f = 10.0

# Resolución
sol = solve_min_time_continuous(α, x_f)

println("Tiempo final óptimo: ", sol.tf)
println("Posiciones: ", sol.x[1,:])
println("Velocidades: ", sol.x[2,:])
println("Control: ", sol.u[1,:])

LoadError: MethodError: no method matching Model(::Type{Ipopt.Optimizer}, ::Int64, ::Int64)

[0mClosest candidates are:
[0m  Model([91m::DataType...[39m)
[0m[90m   @[39m [36mCTBase[39m [90m~/.julia/packages/CTBase/k5wSd/src/[39m[90m[4mmodel.jl:58[24m[39m
